In [2]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from functions import util
import json
import os
import warnings
import requests
warnings.filterwarnings("ignore")

#### util police api

In [26]:

def get_polisens_events(date):
    url = f"https://polisen.se/api/events?DateTime={date}"

    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an error for HTTP errors
        events = response.json()
        return events

    except requests.exceptions.RequestException as e:
        print("Error fetching data:", e)
        return None


def filter_event_type(events, type="Trafikolycka"):
    if events is None:
        return 0
    df = pd.DataFrame(events)
    filtered_events = df[df["type"].str.contains(type, na=False)]
    return filtered_events


def get_daily_events(date = None, type="Trafikolycka"):
    """Get events (accidents often) for a specific date

    Args:
        date ("YYYY-MM-DD" , optional): Defaults to todays date.
        type (str, optional): Type of report to count. Defaults to "Trafikolycka".

    Returns:
        int: Number of reports made on the specified date
    """
    
    if date is None:
        date = datetime.date.today()

    events = get_polisens_events(date)
    filtered_events = filter_event_type(events, type)
    
    return filtered_events


#### Available data 2015-01-07

In [23]:
# Collect the number of accidents for each day from 2024-08-01 until yesterday
start_date = datetime(2024, 8, 1).date()
end_date = datetime.now().date() - timedelta(days=1)
date_range = pd.date_range(start=start_date, end=end_date)

accidents_per_day = []

for single_date in date_range:
    date_str = single_date.strftime("%Y-%m-%d")
    daily_events = get_daily_events(date=date_str)
    accidents_per_day.append(len(daily_events))

# Create a DataFrame with the collected data
accidents_df = pd.DataFrame({
    "date": date_range,
    "accidents": accidents_per_day
})

# Save the DataFrame to a CSV file
accidents_df.to_csv("../data/police_api_backfill.csv", index=False)